In [1]:
# ----------------------------------------#
# N=新产品、新管理阶层、股价新高：选择怎样的入市时机
#! 价格分析：
# 股价新高：用价格捕捉，在价格系统中进行筛选
# 52周新高
# 60天新高
# ----------------------------------------#

In [7]:
import backtrader as bt
import pandas as pd
import numpy as np
# from get_score import get_score_ch
from sqlalchemy import create_engine
import talib as ta
import datetime
import pandas as pd

engine= create_engine('sqlite:///../../dataset/us/us_ticker_price_yf.db')

In [48]:
ticker = 'INTC'
ticker_list = ['INTC','AAPL','GOOG','BABA']


In [52]:
total = pd.DataFrame()
for ticker in ticker_list:
    hist = pd.read_sql(ticker, engine)
    hist.rename(columns={'hgih':'high'},inplace=True)
    # hist['chg_pct'] = ((hist.close - hist.open)/hist.open)*100
    hist['close'] = hist['close'].apply(lambda x: round(x,3))
    # temp = hist.tail(1)
    temp = pd.DataFrame()
    temp['symbol'] = 'symbol'
    temp['datetime'] = hist.tail(1).datetime
    temp.symbol = ticker
    temp.index = range(len(temp))
    # temp = temp[['Open, High, Low, Close, Volume, datetime']
    # 注：为1就是好的
    if hist.iloc[-1].close >= hist.tail(365).close.max() *0.75:
        temp['52-week-high-zone'] = 1
        print('进入52周前高区')
    else: 
        temp['52-week-high-zone'] = 0
    if hist.iloc[-1].close >= hist.tail(365).close.min() * 1.25:
        temp['52-week-low-zone'] = 1
        print('离开52周低价区')
    else:
        temp['52-week-low-zone'] = 0
    if hist.iloc[-1].close > hist.tail(60).close.max() *0.9:
        print('进入60天前高区')
        temp['60-day-high-zone'] = 1
    else:
        temp['60-day-high-zone'] = 0
    if hist.iloc[-1].close > hist.tail(60).close.min() * 1.1:
        print('离开60天低价区')
        temp['60-day-low-zone'] = 1
    else:
        temp['60-day-low-zone'] = 0

    total = pd.concat([total, temp])

进入52周前高区
离开52周低价区
离开60天低价区
离开52周低价区
离开60天低价区


In [43]:

if hist.iloc[-1].close >= hist.tail(365).close.max() *0.75:
    temp['52-week-high-zone'] = 1
    print('进入52周前高区')
else: 
    temp['52-week-high-zone'] = 0
if hist.iloc[-1].close >= hist.tail(365).close.min() * 1.25:
    temp['52-week-low-zone'] = 1
    print('离开52周低价区')
else:
    temp['52-week-low-zone'] = 0
if hist.iloc[-1].close > hist.tail(60).close.max() *0.9:
    print('进入60天前高区')
    temp['60-day-high-zone'] = 1
else:
    temp['60-day-high-zone'] = 0
if hist.iloc[-1].close > hist.tail(60).close.min() * 1.1:
    print('离开60天低价区')
    temp['60-day-low-zone'] = 1
else:
    temp['60-day-low-zone'] = 0

In [44]:
temp

,symbol,datetime,52-week-high-zone,52-week-low-zone,60-day-high-zone,60-day-low-zone
0,INTC,2022-07-14,0,0,0,0


In [45]:
total = pd.concat([total, temp])
total

,symbol,datetime,52-week-high-zone,52-week-low-zone,60-day-high-zone,60-day-low-zone
0,INTC,2022-07-14,0,0,0,0


In [10]:
currenttime = datetime.now()
deltatime = timedelta(day=365)
currentday = str(currenttime.year) + str(currenttime.month) + str(currenttime.day)
currentday

'2022816'

In [37]:
len_ = len(aapl)
for i in range(365,len_):
    if aapl.loc[i].close == aapl.loc[i-365:i].close.max():
        print("date:{} close:{}".format(aapl.loc[i].trade_date,aapl.loc[i].close))

date:20200622 close:358.87
date:20200623 close:366.53
date:20200706 close:373.85
date:20200708 close:381.37
date:20200709 close:382.73
date:20200710 close:383.68
date:20200714 close:388.23
date:20200715 close:390.9
date:20200720 close:393.43
date:20200731 close:425.04
date:20200803 close:435.75
date:20200804 close:438.66
date:20200805 close:440.25
date:20200806 close:455.61
date:20200813 close:460.04
date:20200818 close:462.25
date:20200819 close:462.83
date:20200820 close:473.1
date:20200821 close:497.48
date:20200824 close:503.43
date:20200826 close:506.09


In [1]:
# ----------------------------------------#
#! 价格分析：
# 供应和需求
# 股票的单日成交量是评估供需情况的最佳指标。如果一支股票的价格发生回落，
# 你就必须观察一下交易量是否萎缩。如果交易萎缩，则意味着市场有卖空的压力。
# 如果股票价格上升，你要进一步观察交易量是否有显著扩张，
# 这种情况下很可能是由于机构投资者在买进此种股票建仓的缘故。
#! 进一步查看
# 一旦股票自底部整理形态（见第12章 关于如何阅读股价图以及识别领导股的价格形态）向上突破之后，
# 日成交量应该较正常水准至少高出50%，有的时候甚至可能超过100%或者更高。

# 个人理解：
# 这里强调了两个重点，一个是市场的卖空压力电，也就是我们的卖出点
# 二是成交量的突破，也就是说我们价格的反转点，从这个交易量起市场就要走向上升状态
# 但是这两个点的成立和判断还需要价格的辅助，特别是第一个卖出点的判断，而且这些不属于我们识别股票的阶段，
# 应该是选择买入卖出时机的时候，更加需要这个条件帮助我们去判断
# ----------------------------------------#

In [3]:
# ----------------------------------------#
#! 价格分析
# RS评级是用于比较一支特定股票在过去一年（52周）
# 内相对于市场其他股票的价格表现的一种专利评级。
# 简而言之：大家都跌的时候我跌的少，大家都涨的时候我才能涨得快
# ----------------------------------------#
# RS指标是我个人认为最好的指标之一，其特性使其在任何市场都游刃有余，
# 但是其获取的过程也相对复杂，需要很多的数据作为支持，
# 下面的过程我们将主要的时间和精力都去做出这个指标并且力求和Investor'Daily 的结果相同


In [ ]:
# ----------------------------------------#
# M=市场走向：如何判断大盘走势 Market
# SP500 & Nasdaq
# 要想识别大盘顶点，你就要紧密跟踪每天的标准普尔500、纳斯达克指数。
# 在上涨行情中，必有某一天市场成交总量较前一天会有增加，
# 但是其指数却会出现回落倾向（主要表现是这一天的价格涨幅明显小于前一天的价格涨幅）
# ----------------------------------------#
# 这一部分看似最简单，只需要简单的获取下指数然后算个MA50或MA200就结束了，
# 但是由于权值股的存在，使得SP500并不能很好的展示趋势

In [42]:
import yfinance as yf
mstf = yf.Ticker('MSFT')
mstf.dividends

Date
2003-02-19    0.08
2003-10-15    0.16
2004-08-23    0.08
2004-11-15    3.08
2005-02-15    0.08
              ... 
2020-05-20    0.51
2020-08-19    0.51
2020-11-18    0.56
2021-02-17    0.56
2021-05-19    0.56
Name: Dividends, Length: 70, dtype: float64

In [43]:
mstf.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 221000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related Client Access Licenses; GitHub that provides a collaboration platform and code hosting service for developers; Nuance provides healthcare and enterprise AI solutions; and Azure, a cloud platform. It also offers enterprise